In [1]:
import networkx as nx
G = nx.Graph()

In [2]:
import collections

In [41]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [42]:
def GetTaxiway(FlightNodes, FlightEdges):
    '''
    Gets the Taxiway from Raw Data files showing the nodes and edges in Tuples
    '''
    FlightNodesCoords = list(FlightNodes.split("\n")[3:-1])
    FlightEdgesCoords = list(FlightEdges.split("\n")[3:])
    
    NodeID = []
    NodeXCoords = []
    NodeYCoords = []
    NodeLon = []
    NodeLat = []
    NodeName = []
    NodeType = []
    
    for item in FlightNodesCoords:
        if item == []:
            pass
        else:
            ItemSplit = list(item.split(";")[0:-1])
            try:  
                NodeID.append(ItemSplit[1])
                NodeXCoords.append(ItemSplit[2])
                NodeYCoords.append(ItemSplit[3])
                NodeLat.append(ItemSplit[4])
                NodeLon.append(ItemSplit[5])
                NodeName.append(ItemSplit[6])
                NodeType.append(ItemSplit[7])
            except IndexError:
                break
    FlightNodesList = list(zip(NodeID, NodeLat, NodeLon, NodeName, NodeType))
    
    EdgeID = [] 
    EdgeStart = [] 
    EdgeEnd = [] 
    EdgeDirected = [] 
    EdgeLength = [] 
    EdgeSpecification = [] 
    EdgeTraversalTime = [] 
    EdgeName = [] 
    
    for item in FlightEdgesCoords:
        
        if item == []:
            pass
        else:
            ItemSplit = list(item.split(";"))
            try:
                EdgeID.append(ItemSplit[1])
                EdgeStart.append(ItemSplit[2])
                EdgeEnd.append(ItemSplit[3])
                EdgeDirected.append(ItemSplit[4])
                EdgeLength.append(ItemSplit[5])
                EdgeSpecification.append(ItemSplit[6])
                EdgeTraversalTime.append(ItemSplit[7])
                EdgeName.append(ItemSplit[8])
            except IndexError:
                break
    FlightEdgesList = list(zip(EdgeID, EdgeStart, EdgeEnd, EdgeDirected, EdgeLength, EdgeSpecification, EdgeName))
    FlightEdgesWeights = list(zip(EdgeID, EdgeStart, EdgeEnd, EdgeLength))
    
    return (FlightNodesList, FlightEdgesList, FlightEdgesWeights)

In [43]:
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]

In [55]:
#This will unpack each tuple to call any information about the list of Edges and Nodes
def GetEdgeID(FlightEdgesList):
    return [x[0] for x in FlightEdgesList]

def GetEdgeStart(FlightEdgesList, FlightNodesList):
    FlightEdgesStartNodes = [x[1] for x in FlightEdgesList]
    index = 0
    EdgeStartPoint = []
    for item in [x[1] for x in FlightEdgesList]:
        counter = 0
        for value in [x[0] for x in FlightNodesList]:
        
            if item == value:
                EdgeStartPoint.append((FlightNodesList[counter][1], FlightNodesList[counter][2]))
            counter += 1
        index += 1
    return (EdgeStartPoint)

def GetEdgeEnd(FlightEdgesList, FlightNodesList):
    FlightEdgesEndNodes = [x[2] for x in FlightEdgesList]
    index = 0
    EdgeEndPoint = []
    for item in [x[2] for x in FlightEdgesList]:
        counter = 0
        for value in [x[0] for x in FlightNodesList]:
        
            if item == value:
                EdgeEndPoint.append((FlightNodesList[counter][1], FlightNodesList[counter][2]))
            counter += 1
        index += 1
    return (EdgeEndPoint)

def GetEdgeLength(FlightEdgesList):
    return [x[4] for x in FlightEdgesList]

def GetEdgeSpecification(FlightEdgesList):
    return [x[5] for x in FlightEdgesList]


In [48]:
GetEdgeStart(FlightEdgesList, FlightNodesList);

In [47]:
[x[1] for x in FlightEdgesList];

In [49]:
#Now that we have our data in Tuples, we can start making the Node Map with dictionaries

def GetNodeLinks(FlightNodes, FlightEdges):
    '''
    Returns Nodes with edges connected on a Graph in a dictionary
    '''
    Graph = {}
    for item in FlightNodes:
        Graph[item[0]] = []
        
    for item in FlightEdges:
        
        if item[1] in Graph.keys():
            Graph[item[1]].append(item[2])
        if item[2] in Graph.keys():
            Graph[item[2]].append(item[1])
    
    return (Graph)

In [52]:
BabyFlightLinks = GetNodeLinks(FlightNodesList, FlightEdgesList)
BabyFlightLinks;

In [54]:
FlightEdgesList

[('40', '1527', '1526', '0', '28.116986952451075', 'taxiway', ''),
 ('41', '1936', '329', '0', '78.58175488063434', 'taxiway', ''),
 ('42', '329', '331', '0', '78.06647871946844', 'taxiway', ''),
 ('43', '331', '330', '0', '72.84067042368781', 'taxiway', ''),
 ('44', '330', '328', '0', '73.33289721729203', 'taxiway', ''),
 ('45', '328', '2040', '0', '119.82890859132637', 'taxiway', ''),
 ('46', '2040', '327', '0', '48.74226221437479', 'taxiway', ''),
 ('47', '327', '2039', '0', '10.458676393213096', 'taxiway', ''),
 ('50', '326', '325', '0', '15.501521996958708', 'taxiway', ''),
 ('53', '971', '324', '0', '7.306940109651343', 'taxiway', ''),
 ('54', '324', '968', '0', '5.558682621325132', 'taxiway', ''),
 ('55', '968', '2038', '0', '45.78079769278536', 'taxiway', ''),
 ('59', '1918', '377', '0', '11.962819189763287', 'taxiway', ''),
 ('68', '1098', '1103', '0', '4.877345915554401', 'taxiway', ''),
 ('76', '1928', '282', '0', '8.996283349840507', 'taxiway', ''),
 ('77', '1940', '294', '

In [13]:
def JunctionCheck(FlightEdge, BabyFlightLinks):
    pass

In [63]:
for index, edge in enumerate(FlightEdgesList):
        EdgeID = GetEdgeID(FlightEdgesList)[index]
        EdgeStart = GetEdgeStart(FlightEdgesList, FlightNodesList)[index]
        EdgeEnd = GetEdgeEnd(FlightEdgesList, FlightNodesList)[index]
        EdgeLength = GetEdgeLength(FlightEdgesList)[index]
        EdgeType = GetEdgeSpecification(FlightEdgesList)[index]

        StartX = float(EdgeStart[0])
        StartY = float(EdgeStart[1])
        EndX = float(EdgeEnd[0])
        EndY = float(EdgeEnd[1])
        MidPointX = (StartX + (EndX - StartX)/2)
        MidPointY = (StartY +(EndY - StartY)/2)
        EdgeMidPoints = list((EdgeID, MidPointX, MidPointY))

KeyboardInterrupt: 

In [ ]:
EdgeMidPoints